# Limpieza del dataset "charts" para visualización en Tableau.

In [36]:
from pathlib import Path
import pandas as pd, numpy as np
pd.set_option('display.max_columns', 100)
DATA_DIR = Path.cwd().parent / "data" 
SPOTY = DATA_DIR / 'charts.csv'

if not SPOTY.exists():
    raise FileNotFoundError(f"Training file not found at {TRAIN_CSV}. Please place cs-training.csv in data/ and re-run.")

charts = pd.read_csv(SPOTY)

print('Datos cargados correctamente.')

Datos cargados correctamente.


**Explicación:** En esta celda se cargará el .csv "charts" (el más grande de los propuestos) para modelizarlo a nuestro gusto y mantener la información que nos resulte relevante.

In [37]:
charts.info()
charts.head(7)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26173514 entries, 0 to 26173513
Data columns (total 9 columns):
 #   Column   Dtype  
---  ------   -----  
 0   title    object 
 1   rank     int64  
 2   date     object 
 3   artist   object 
 4   url      object 
 5   region   object 
 6   chart    object 
 7   trend    object 
 8   streams  float64
dtypes: float64(1), int64(1), object(7)
memory usage: 1.8+ GB


,title,rank,date,artist,url,region,chart,trend,streams
0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,top200,SAME_POSITION,253019.0
1,Vente Pa' Ca (feat. Maluma),2,2017-01-01,Ricky Martin,https://open.spotify.com/track/7DM4BPaS7uofFul...,Argentina,top200,MOVE_UP,223988.0
2,Reggaetón Lento (Bailemos),3,2017-01-01,CNCO,https://open.spotify.com/track/3AEZUABDXNtecAO...,Argentina,top200,MOVE_DOWN,210943.0
3,Safari,4,2017-01-01,"J Balvin, Pharrell Williams, BIA, Sky",https://open.spotify.com/track/6rQSrBHf7HlZjtc...,Argentina,top200,SAME_POSITION,173865.0
4,Shaky Shaky,5,2017-01-01,Daddy Yankee,https://open.spotify.com/track/58IL315gMSTD37D...,Argentina,top200,MOVE_UP,153956.0
5,Traicionera,6,2017-01-01,Sebastian Yatra,https://open.spotify.com/track/5J1c3M4EldCfNxX...,Argentina,top200,MOVE_DOWN,151140.0
6,Cuando Se Pone a Bailar,7,2017-01-01,Rombai,https://open.spotify.com/track/1MpKZi1zTXpERKw...,Argentina,top200,MOVE_DOWN,148369.0


**Explicación:** Observamos la información principal y la cabecera del dataset original.

In [38]:
charts = charts.drop(columns=['trend', 'url', 'chart'])

**Explicación:** Eliminar las columnas 'trend', 'url' y 'chart' ya que no consideramos que nos vayan a ser útiles en las representaciones a realizar.

In [39]:
charts['date'] = charts['date'].astype(str).str[5:7]
charts.head()

charts['date'] = pd.to_numeric(charts['date']).astype('int8')
print(charts['date'].dtype)
charts.head()

int8


,title,rank,date,artist,region,streams
0,Chantaje (feat. Maluma),1,1,Shakira,Argentina,253019.0
1,Vente Pa' Ca (feat. Maluma),2,1,Ricky Martin,Argentina,223988.0
2,Reggaetón Lento (Bailemos),3,1,CNCO,Argentina,210943.0
3,Safari,4,1,"J Balvin, Pharrell Williams, BIA, Sky",Argentina,173865.0
4,Shaky Shaky,5,1,Daddy Yankee,Argentina,153956.0


**Explicación:** Sobreescribimos la columna 'date' quedándonos solo con los caracteres del mes (1 para enero, 2 para febrero etc.) y verificamos los cambios. Además, convertimos la columna a números enteros (usamos 'int8' porque los meses, al ser 12, caben de sobra en este tipo de dato pequeño y porque así hacemos lo más sencillo posible el dataset para evitar futuros errores a la hora de cargarlo a Tableau).

In [40]:
charts['streams'] = charts['streams'].fillna(0).astype('int64')

print("Nuevo tipo de dato en streams:", charts['streams'].dtype)
charts.info()
charts.head()

Nuevo tipo de dato en streams: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26173514 entries, 0 to 26173513
Data columns (total 6 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   title    object
 1   rank     int64 
 2   date     int8  
 3   artist   object
 4   region   object
 5   streams  int64 
dtypes: int64(2), int8(1), object(3)
memory usage: 1023.4+ MB


,title,rank,date,artist,region,streams
0,Chantaje (feat. Maluma),1,1,Shakira,Argentina,253019
1,Vente Pa' Ca (feat. Maluma),2,1,Ricky Martin,Argentina,223988
2,Reggaetón Lento (Bailemos),3,1,CNCO,Argentina,210943
3,Safari,4,1,"J Balvin, Pharrell Williams, BIA, Sky",Argentina,173865
4,Shaky Shaky,5,1,Daddy Yankee,Argentina,153956


**Explicación:** Rellenamos posibles huecos con 0 en la variable 'streams' y convertimos a entero de 64 bits (int64). Esto elimina el decimal, ahorrando así memoria. A pesar de que esta acción pueda generar algún error muy simple en el futuro derivado de alguna incongruencia en la propia base de datos original (como una canción con 0 streams en una región y mes determinados), se nos ha comunicado por parte del profesor que la parte de limpieza y modelado no es para nada la principal de este trabajo, y que errores de este calibre no son determinantes dado el objetivo de visualización de dicha práctica, por lo que continuaremos con la limpieza sin tener esto en cuenta.

In [41]:
if 'rank' in charts.columns:
    charts = charts.drop(columns=['rank'])
    
charts_agrupado = charts.groupby(['title', 'artist', 'region', 'date'])['streams'].sum().reset_index()

print(f"Filas originales: {len(charts)}")
print(f"Filas tras agrupar: {len(charts_agrupado)}")

charts_agrupado.head()

Filas originales: 26173514
Filas tras agrupar: 1371659


,title,artist,region,date,streams
0,!,O.S.T.R.,Poland,2,40671
1,!,O.S.T.R.,Poland,3,83043
2,!,"Samey, Gleb",Czech Republic,9,53759
3,!,"Samey, Gleb",Czech Republic,10,3098
4,!,"Samey, Gleb",Slovakia,1,57570


**Explicación:** Tras pensarlo mejor eliminamos la columna 'rank' ya que vamos a realizar unas agrupaciones que harán que esta sea inservible y procedemos a agrupar por Título, Artista, Región y Fecha (Mes), calculando así la suma de 'streams' para cada grupo.

**Observación:** Aunque pueda parecer un error que las primeras canciones tengan un símbolo de exclamación, es simplemente el título de varias canciones listadas en los tops.

In [42]:
mapa_meses = {
    1: 'Enero', 2: 'Febrero', 3: 'Marzo', 4: 'Abril',
    5: 'Mayo', 6: 'Junio', 7: 'Julio', 8: 'Agosto',
    9: 'Septiembre', 10: 'Octubre', 11: 'Noviembre', 12: 'Diciembre'
}

charts_agrupado['date'] = charts_agrupado['date'].map(mapa_meses)
charts_agrupado.head()

,title,artist,region,date,streams
0,!,O.S.T.R.,Poland,Febrero,40671
1,!,O.S.T.R.,Poland,Marzo,83043
2,!,"Samey, Gleb",Czech Republic,Septiembre,53759
3,!,"Samey, Gleb",Czech Republic,Octubre,3098
4,!,"Samey, Gleb",Slovakia,Enero,57570


**Explicación:** Cambiamos el campo mensual a texto asignando a cada mes su respectivo nombre (para optimizar futura interpretabilidad en Tableau por parte de un usuario).

In [43]:
OUTPUT_FILE = DATA_DIR / 'charts_grouped.csv'

print(f"Guardando dataset de {len(charts_agrupado)} filas...")

charts_agrupado.to_csv(OUTPUT_FILE, index=False, encoding='utf-8')

print(f"Archivo guardado en: {OUTPUT_FILE}")

Guardando dataset de 1371659 filas...
Archivo guardado en: C:\Users\jorge\Downloads\Universidad\Cursos\3 carrera\BDA\Visualizacion\data\charts_grouped.csv


**Explicación:** Guardamos el nuevo dataset como .csv para exportar a Tableau y comenzar con la parte visual.